In [ ]:
#install the dash library
pip install dash dash-core-components dash-html-components plotly 

In [3]:
# Dash is a Python framework for building interactive web applications.
import dash 
# dcc (Dash Core Components) contains components like Graph, Dropdown, Slider, etc.
from dash import dcc, html, Input, Output 
# Plotly Express is a plotting library that makes it easy to create plots from DataFrames.
import plotly.express as px  
# Pandas is a data manipulation library, used here for reading and handling the dataset.
import pandas as pd

# Load your dataset here
df = pd.read_csv('qb_pbp_epa2.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Dropdown options for quarterbacks
quarterbacks = df['passer'].unique()
qb_options = [{'label': qb, 'value': qb} for qb in quarterbacks]

# Slider options for seasons
seasons = df['season'].unique()
seasons.sort()

# Define the app layout
app.layout = html.Div([
    html.H1("NFL Quarterback Interactive Dashboard"),
    
    # Dropdown for quarterback selection
    html.Label("Select a Quarterback:"),
    dcc.Dropdown(
        id='qb-dropdown',
        options=qb_options,
        value=quarterbacks[0]  # Set default value
    ),
    
    # Line Chart Placeholder for the selected quarterback
    dcc.Graph(id='line-chart'),
    
    # Slider for season selection
    html.Label("Select a Season:"),
    dcc.Slider(
        id='season-slider',
        min=seasons.min(),
        max=seasons.max(),
        value=seasons.max(),
        marks={str(season): str(season) for season in seasons},
        step=None
    ),
    
    # Bar Chart and Scatter Plot Placeholder for the selected season
    dcc.Graph(id='bar-chart'),
    dcc.Graph(id='scatter-plot')
])

# Callback for updating the line chart based on selected quarterback
@app.callback(
    Output('line-chart', 'figure'),
    [Input('qb-dropdown', 'value')]
)
def update_line_chart(selected_qb):
    # Filter the dataframe for the selected quarterback
    filtered_df = df[df['passer'] == selected_qb]
     # Create and return the line chart figure
    fig = px.line(filtered_df, x="season", y="epa_mean", title=f"{selected_qb}'s Mean EPA Over Seasons")
    return fig

# Callback for updating the bar chart and scatter plot based on selected season
@app.callback(
    [Output('bar-chart', 'figure'),
     Output('scatter-plot', 'figure')],
    [Input('season-slider', 'value')]
)
def update_charts(selected_season):
    # Filter the dataframe for the selected season
    filtered_df = df[df['season'] == selected_season]
    # Create and return the bar chart figure
    bar_fig = px.bar(filtered_df, x="passer", y="epa_mean", title=f"Quarterbacks' Mean EPA in {selected_season}")
    # Create and return the scatter plot figure
    scatter_fig = px.scatter(filtered_df, x="epa_mean", y="pass_td_last", color="passer",
                             title=f"Relationship between Mean EPA and Touchdowns in {selected_season}")
    return bar_fig, scatter_fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8053)     # Start the server on port 8053
